In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline 

In [2]:
path_to_data = '/mnt/storage/HRV/data_atfer_Nekit_skripts/'

# ожидается, что внутри папки, прописанной в пути будут
# selected_mod.dta
# папка code_string/ с записанными в ней кодограмами вида gidn.txt
# папка R-T_code_with_markup/ с записанными в ней R-T сигналами вида gidn.npy (в этом формате сохраняет np:save)
# можно поменять на свое расположение и формат, изменив строки 13, 15, 39, 40 в тесте 1
# и строки 8, 10, 28, 30 в тесте 2
# есть два способа загрузки данных
# считывание кодограммы
# if os.path.isfile(path_to_data + "code_string/" + str(gidn_i) + '.txt'):   
#     trigramm_i = data2trigramm(mode = 'file', path_to_codegramm = path_to_data + "code_string/" + str(gidn_i) + '.txt')
#     trigramm.append(trigramm_i)
# считывание R, T сигналов и вычисление кодограммы, триграмм
# if os.path.isfile(path_to_data + "R-T_code_with_markup/" + str(gidn_i) + '.npy'):
#     data = np.load(path_to_data + 'R-T_code_with_markup/' + str(gidn_i) + '.npy')
#     R = data[0]
#     T = data[1]
#     trigramm.append(data2trigramm(R_T2codegramm(R, T)))

In [3]:
str2y = {'No':0, 'Have now':1, 'Have had':1}
str2num = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5}
#перевод триграммы в число

def trigram2num (string):
    num = 0
    for i, char in enumerate(string):
        num += str2num[char]*(6**i)
    
    return num

In [68]:
# перевод R-T в кодограмму

def R_T2codegramm(R, T):
    
    a = np.arctan(R/T)
    dT = T[1:] - T[:-1]
    dR = R[1:] - R[:-1]
    da = a[1:] - a[:-1]

    codegramm = []
    for i, dt in enumerate(dT):
        if dt > 0 and dR[i] > 0 and da[i] > 0:
            codegramm.append('A')
        elif dt <= 0 and dR[i] <= 0 and da[i] > 0:
            codegramm.append('B')
        elif dt <=0 and dR[i] > 0 and da[i] > 0:
            codegramm.append('C')
        elif dt > 0 and dR[i] <= 0 and da[i] <= 0:
            codegramm.append('D')
        elif dt > 0 and dR[i] > 0 and da[i] <= 0:
            codegramm.append('E')
        elif dt <= 0 and dR[i] <= 0 and da[i] <= 0:
            codegramm.append('F')
        elif T[i] ==  0 or T[i+1] == 0:
            continue
        else:
            print ('ups...', i, 'dt', dt, 'dR', dR[i], 'da', da[i])
            
    return ''.join(codegramm)


In [5]:
# перевод кодограммы в триграммы. принимает на вход кодограмму или файл, в котором она сохранена

def data2trigramm(codegramm = '', mode = 'R-T', path_to_codegramm = ''):
    
    if mode == 'R-T':
        ngramm = CountVectorizer(analyzer='char', ngram_range=(3, 3))
        analyze = ngramm.build_analyzer()
        words = analyze(codegramm)

        trigramm = np.zeros(6**3)
        for i in words:
            trigramm[trigram2num(i)] += 1

        trigramm = trigramm/sum(trigramm)
    elif mode == 'file':
        
        ngramm = CountVectorizer(input = 'file', analyzer='char', ngram_range=(3, 3))
        analyze = ngramm.build_analyzer()
        with open(path_to_codegramm, "r") as o:
            words = analyze(o)

        trigramm = np.zeros(6**3)
        for i in words:
            trigramm[trigram2num(i)] += 1

        trigramm = trigramm/sum(trigramm)

    return trigramm

In [102]:
# первый тест из статьи воронцова, нужно перемешать внутри сигнала кардиоциклы и посчитать триграммы.
# триграммы посчитанные по исходному сигналу должны отличаться от вычесленных по перемешанному.

def test1(gidn, path_to_data, K, fi): 
    
    selected_pp = pd.read_stata(path_to_data + 'selected_mod.dta')

    F_w = []
    B_w = []
    for i in range(K):
        trigramm = []
        for gidn_i in gidn: #800 signals ~ 10 min
            if os.path.isfile(path_to_data + "R-T_code_with_markup/" + str(gidn_i) + '.npy'):

                data = np.load(path_to_data + 'R-T_code_with_markup/' + str(gidn_i) + '.npy')
                np.random.shuffle(data.T)
            
                fi = 1.0/(len(data[0]) - 3)
                R = data[0]
                T = data[1]

                trigramm.append(data2trigramm(R_T2codegramm(R, T)))

        trigramm = np.array(trigramm)
        F_w_i = np.mean(trigramm, axis = 0)
        F_w.append(F_w_i)
        trigramm[trigramm < fi] = 0
        B_w_i = np.mean(trigramm, axis = 0)
        B_w.append(B_w_i)

    F_w = np.array(F_w)
    B_w = np.array(B_w)   

#  __________________________________________________________________________________________________________

    for gidn_i in gidn:
        trigramm = []
        if os.path.isfile(path_to_data + "R-T_code_with_markup/" + str(gidn_i) + '.npy'):

            data = np.load(path_to_data + 'R-T_code_with_markup/' + str(gidn_i) + '.npy')
            R = data[0]
            T = data[1]

            trigramm.append(data2trigramm(R_T2codegramm(R, T)))

    trigramm = np.array(trigramm)
    F_w_orig = np.mean(trigramm, axis = 0)
    trigramm[trigramm < fi] = 0
    B_w_orig = np.mean(trigramm, axis = 0)
    
# ________________________________________________________________________________________________________________
# тут уже подсчитанные признаки и критерий информативности триграмм. можно возвращать его

#     res1 = np.vstack((B_w_orig - np.min(B_w, axis = 0) < 0, np.max(B_w, axis = 0) - B_w_orig < 0)).T.any(axis=1)

#     res2 = np.vstack((F_w_orig - np.min(F_w, axis = 0) < 0, np.max(F_w, axis = 0) - F_w_orig < 0)).T.any(axis=1)

#     DF = (2*F_w_orig - np.min(F_w, axis = 0) - np.max(F_w, axis = 0))/(np.max(F_w, axis = 0) - np.min(F_w, axis = 0))
#     res3 = np.vstack((DF > 1, DF < -1)).T.any(axis=1)

#     DB = (2*B_w_orig - np.min(B_w, axis = 0) - np.max(B_w, axis = 0))/(np.max(B_w, axis = 0) - np.min(B_w, axis = 0))
#     res4 = np.vstack((DB > 1, DB < -1)).T.any(axis=1)

#     res_all = np.vstack((res1, res2, res3, res4)).T.any(axis=1)
    
    return (F_w, F_w_orig, B_w, B_w_orig)

In [119]:
def test2(gidn, gidn_ill, path_to_data, K, fi):
    F_w = []
    B_w = []
    for i in range(K):
        random_gidn = np.random.choice(gidn, 10) #10 - количество случайно взятых гидн из выборки
        trigramm = []
        for gidn_i in random_gidn:
            if os.path.isfile(path_to_data + "code_string/" + str(gidn_i) + '.txt'):
                
                trigramm_i = data2trigramm(mode = 'file', path_to_codegramm = path_to_data + "code_string/" + str(gidn_i) + '.txt')
                trigramm.append(trigramm_i)
                fi = 1.0/(len(data[0]) - 3)

        trigramm = np.array(trigramm)
        F_w_i = np.mean(trigramm, axis = 0)
        F_w.append(F_w_i)
        trigramm[trigramm < fi] = 0
        B_w_i = np.mean(trigramm, axis = 0)
        B_w.append(B_w_i)

    F_w = np.array(F_w)
    B_w = np.array(B_w)
    
# _________________________________________________________________________________________________________________

    X = []
    for gidn_i in gidn_ill:
        trigramm = []
        if os.path.isfile(path_to_data + "code_string/" + str(gidn_i) + '.txt'):
            
            trigramm_i = data2trigramm(mode = 'file', path_to_codegramm = path_to_data + "code_string/" + str(gidn_i) + '.txt')
            trigramm.append(trigramm_i)

    trigramm = np.array(trigramm)
    F_w_orig = np.mean(trigramm, axis = 0)
    trigramm[trigramm < fi] = 0
    B_w_orig = np.mean(trigramm, axis = 0)
# ________________________________________________________________________________________________________________
# тут уже подсчитанные признаки и критерий информативности триграмм. можно возвращать его

#     res1 = np.vstack((B_w_orig - np.min(B_w, axis = 0) < 0, np.max(B_w, axis = 0) - B_w_orig < 0)).T.any(axis=1)

#     res2 = np.vstack((F_w_orig - np.min(F_w, axis = 0) < 0, np.max(F_w, axis = 0) - F_w_orig < 0)).T.any(axis=1)

#     DF = (2*F_w_orig - np.min(F_w, axis = 0) - np.max(F_w, axis = 0))/(np.max(F_w, axis = 0) - np.min(F_w, axis = 0))
#     res3 = np.vstack((DF > 1, DF < -1)).T.any(axis=1)

#     DB = (2*B_w_orig - np.min(B_w, axis = 0) - np.max(B_w, axis = 0))/(np.max(B_w, axis = 0) - np.min(B_w, axis = 0))
#     res4 = np.vstack((DB > 1, DB < -1)).T.any(axis=1)

    return (F_w, F_w_orig, B_w, B_w_orig)

In [108]:
files = os.listdir(path_to_data + "R-T_code_with_markup/")
gidn = [i[:-4] for i in files[:10]]

In [120]:
(F_w, F_w_orig, B_w, B_w_orig) = test2(gidn, gidn, path_to_data, 2, 0.001)